<a href="https://colab.research.google.com/github/Steinspass/Fruit_Classifier/blob/master/fruits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')


base_dir = '/content/drive/My Drive/fruits-360/Training'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_generator = datagen.flow_from_directory(
    base_dir, 
    target_size =(IMAGE_SIZE, IMAGE_SIZE),
    batch_size  = BATCH_SIZE,
    subset = 'training'
    )

val_generator = datagen.flow_from_directory(
    base_dir, 
    target_size =(IMAGE_SIZE, IMAGE_SIZE),
    batch_size  = BATCH_SIZE,
    subset = 'validation'
    )

Found 67692 images belonging to 131 classes.
Found 0 images belonging to 131 classes.


In [4]:
print(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.text', 'w') as f:
  f.write(labels)

{'Apple Braeburn': 0, 'Apple Crimson Snow': 1, 'Apple Golden 1': 2, 'Apple Golden 2': 3, 'Apple Golden 3': 4, 'Apple Granny Smith': 5, 'Apple Pink Lady': 6, 'Apple Red 1': 7, 'Apple Red 2': 8, 'Apple Red 3': 9, 'Apple Red Delicious': 10, 'Apple Red Yellow 1': 11, 'Apple Red Yellow 2': 12, 'Apricot': 13, 'Avocado': 14, 'Avocado ripe': 15, 'Banana': 16, 'Banana Lady Finger': 17, 'Banana Red': 18, 'Beetroot': 19, 'Blueberry': 20, 'Cactus fruit': 21, 'Cantaloupe 1': 22, 'Cantaloupe 2': 23, 'Carambula': 24, 'Cauliflower': 25, 'Cherry 1': 26, 'Cherry 2': 27, 'Cherry Rainier': 28, 'Cherry Wax Black': 29, 'Cherry Wax Red': 30, 'Cherry Wax Yellow': 31, 'Chestnut': 32, 'Clementine': 33, 'Cocos': 34, 'Corn': 35, 'Corn Husk': 36, 'Cucumber Ripe': 37, 'Cucumber Ripe 2': 38, 'Dates': 39, 'Eggplant': 40, 'Fig': 41, 'Ginger Root': 42, 'Granadilla': 43, 'Grape Blue': 44, 'Grape Pink': 45, 'Grape White': 46, 'Grape White 2': 47, 'Grape White 3': 48, 'Grape White 4': 49, 'Grapefruit Pink': 50, 'Grapefrui

In [5]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape= IMG_SHAPE,
    include_top = False,
    weights = 'imagenet'
)


9412608/9406464 [==============================] - 0s 0us/step


In [6]:
base_model.trainable = False

In [7]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(131, activation='softmax')
])

In [8]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [10]:
epochs = 10

history = model.fit(
    train_generator, 
    epochs=epochs,
    steps_per_epoch=100
    #validation_data = val_generator
)


Epoch 1/10
100/100 [==============================] - 5069s 51s/step - loss: 3.7514 - accuracy: 0.1928
Epoch 2/10
100/100 [==============================] - 4521s 45s/step - loss: 1.8232 - accuracy: 0.5480
Epoch 3/10
100/100 [==============================] - 4106s 41s/step - loss: 0.9967 - accuracy: 0.7184
Epoch 4/10
100/100 [==============================] - 3770s 38s/step - loss: 0.6864 - accuracy: 0.7997
Epoch 5/10
100/100 [==============================] - 3487s 35s/step - loss: 0.5760 - accuracy: 0.8253
Epoch 6/10
100/100 [==============================] - 3206s 32s/step - loss: 0.4834 - accuracy: 0.8489
Epoch 7/10
100/100 [==============================] - 2829s 28s/step - loss: 0.3881 - accuracy: 0.8822
Epoch 8/10
100/100 [==============================] - 2597s 26s/step - loss: 0.3718 - accuracy: 0.8836
Epoch 9/10
100/100 [==============================] - 2346s 23s/step - loss: 0.3419 - accuracy: 0.8905
Epoch 10/10
100/100 [==============================] - 2080s 21s/step - l

In [11]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: assets


In [12]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: ignored